<a href="https://colab.research.google.com/github/dkmachinelearning/dkmachinelearning/blob/main/KERAS_EXAMPLES/PIMA_Indians_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt
import tensorflow as tf
import tensorflow_datasets as tfds



In [4]:
(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()
train_ds = tf.data.Dataset.from_tensor_slices((img_train/255.0, label_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((img_test/255.0, label_test)).batch(32)



29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
inputs = tf.keras.Input(shape=(28, 28), batch_size=32)
flatten_layer = tf.keras.layers.Flatten()(inputs)
dense = tf.keras.layers.Dense(units=512, activation='relu')(flatten_layer)
outputs = tf.keras.layers.Dense(units=10)(dense)



In [6]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [7]:
def build_model(hp):
    dnn_layers_ss = [1,2,3]
    dnn_units_min, dnn_units_max = 32, 512
    dropout_ss = [0.1, 0.2]
    active_func_ss = ['relu', 'tanh']
    optimizer_ss = ['adam']
    lr_min, lr_max = 1e-4, 1e-1

    active_func = hp.Choice('activation', active_func_ss)
    optimizer = hp.Choice('optimizer', optimizer_ss)
    lr = hp.Float('learning_rate', min_value=lr_min, max_value=lr_max, sampling='log')

    inputs = tf.keras.Input(shape=(28,28))
    flatten_layer = tf.keras.layers.Flatten()(inputs)

    # create hidden layers
    dnn_units = hp.Int(f"0_units", min_value=dnn_units_min, max_value=dnn_units_max)
    dense = tf.keras.layers.Dense(units=dnn_units, activation=active_func)(flatten_layer)
    for layer_i in range(hp.Choice("n_layers", dnn_layers_ss) - 1):
        dnn_units = hp.Int(f"{layer_i}_units", min_value=dnn_units_min, max_value=dnn_units_max)
        dense = tf.keras.layers.Dense(units=dnn_units, activation=active_func)(dense)
        if hp.Boolean("dropout"):
            dense = tf.keras.layers.Dropout(rate=0.25)(dense)
    outputs = tf.keras.layers.Dense(units=10)(dense)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "SGD":
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    else:
        raise("Not supported optimizer")

    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [12]:
def build_tuner(model, hpo_method, objective, dir_name):
    if hpo_method == "RandomSearch":
        tuner = kt.RandomSearch(model, objective=objective, max_trials=3, executions_per_trial=1,
                               project_name=hpo_method, directory=dir_name)
    elif hpo_method == "Hyperband":
        tuner = kt.Hyperband(model, objective=objective, max_epochs=3, executions_per_trial=1,
                            project_name=hpo_method, directory=dir_name)
    elif hpo_method == "BayesianOptimization":
        tuner = kt.BayesianOptimization(model, objective=objective, max_trials=3, executions_per_trial=1,
                                       project_name=hpo_method, directory=dir_name)
    return tuner

obj = kt.Objective('val_accuracy', direction='max')
dir_name = "v1"
randomsearch_tuner = build_tuner(build_model, "Hyperband", obj, dir_name)
randomsearch_tuner.search(train_ds, epochs=3, validation_data=test_ds)

Trial 6 Complete [00h 01m 58s]
val_accuracy: 0.8392999768257141

Best val_accuracy So Far: 0.8715999722480774
Total elapsed time: 00h 05m 46s


In [ ]:
total_ds = train_ds.concatenate(test_ds)
best_model = build_model(randomsearch_tuner.get_best_hyperparameters(1)[0])
best_model.fit(total_ds, epochs=10)

Epoch 1/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 36s 16ms/step - accuracy: 0.7845 - loss: 0.6098
Epoch 2/10
 481/2188 ━━━━━━━━━━━━━━━━━━━━ 26s 15ms/step - accuracy: 0.8627 - loss: 0.3626